In [ ]:
# !pip install datasets
# !pip install evaluate

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("mteb/tweet_sentiment_extraction")

README.md:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/3.63M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/465k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [4]:
import pandas as pd

df = pd.DataFrame(dataset['train'])

In [5]:
df

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,0,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,0,negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,2,positive
27479,ed167662a5,But it was worth it ****.,2,positive


In [6]:
from transformers import GPT2Tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [9]:
def tokenize_texts(inputs):
  return tokenizer(inputs['text'], padding="max_length", truncation=True)

In [10]:
tokenize_data = dataset.map(tokenize_texts,batched=True)

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [11]:
train_data = tokenize_data['train'].shuffle(seed=44).select(range(1000))
eval_data = tokenize_data['test'].shuffle(seed=44).select(range(1000))

In [12]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import evaluate

metric = evaluate.load("accuracy")

In [14]:
import numpy as np
def evaluation(acc):
  logits, labels = acc
  predictions = np.argmax(logits, axis = -1)
  return metric.compute(predictios=predictions,refrences=labels)

In [15]:
from transformers import TrainingArguments, Trainer

train_arg = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4
)

In [16]:
trainer = Trainer(
    model=model,
    args=train_arg,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=evaluation
)

In [17]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,3.766800


TrainOutput(global_step=750, training_loss=3.2518545735677082, metrics={'train_runtime': 851.2744, 'train_samples_per_second': 3.524, 'train_steps_per_second': 0.881, 'total_flos': 1567794659328000.0, 'train_loss': 3.2518545735677082, 'epoch': 3.0})

In [20]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("test_trainer")
tokenizer.save_pretrained("test_trainer")

('test_trainer/tokenizer_config.json',
 'test_trainer/special_tokens_map.json',
 'test_trainer/vocab.json',
 'test_trainer/merges.txt',
 'test_trainer/added_tokens.json')

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

model_path = "/content/test_trainer"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2ForSequenceClassification.from_pretrained(model_path)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [26]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=-1).item()

    return predicted_label, torch.softmax(logits, dim=-1).tolist()

sample_text = "what's going on"
predicted_label, probabilities = predict_sentiment(sample_text)

print(f"Input Text: {sample_text}")
print(f"Predicted Label: {predicted_label}")
print(f"Probabilities: {probabilities}")


Input Text: what's going on
Predicted Label: 1
Probabilities: [[0.018183259293437004, 0.9511358141899109, 0.030680881813168526]]


In [27]:
# !pip install huggingface_hub

In [29]:
from huggingface_hub import notebook_login
notebook_login()

In [30]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

model = GPT2ForSequenceClassification.from_pretrained("/content/test_trainer")
tokenizer = GPT2Tokenizer.from_pretrained("/content/test_trainer")

model.push_to_hub("gpt2-sentiment-analysis-tweets")
tokenizer.push_to_hub("gpt2-sentiment-analysis-tweets")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/riturajpandey739/gpt2-sentiment-analysis-tweets/commit/e0730a8b76a2033c9b097ef5d99619094414c637', commit_message='Upload tokenizer', commit_description='', oid='e0730a8b76a2033c9b097ef5d99619094414c637', pr_url=None, repo_url=RepoUrl('https://huggingface.co/riturajpandey739/gpt2-sentiment-analysis-tweets', endpoint='https://huggingface.co', repo_type='model', repo_id='riturajpandey739/gpt2-sentiment-analysis-tweets'), pr_revision=None, pr_num=None)